In [1]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [2]:
def buscar_productos_decathlon():
    """
    Realiza una búsqueda de productos en el sitio web de Decathlon y recopila información sobre los primeros 5 resultados.

    Returns:
    dict: Un diccionario con información sobre los productos encontrados, incluyendo nombres, precios y puntuaciones.
    """
    
    # Inicializar el controlador de Chrome
    driver = webdriver.Chrome()

    # Maximizar la ventana del navegador
    driver.maximize_window()

    # Acceder al sitio web de Decathlon
    driver.get("https://www.decathlon.es/")

    # Esperar 3 segundos para cargar la página
    sleep(3)

    # Hacer clic en el botón de aceptar las cookies utilizando su selector CSS
    driver.find_element("css selector", "#didomi-notice-agree-button").click()

    # Solicitar al usuario que ingrese qué producto desea buscar
    eleccion_usuario = input("¿Qué quieres buscar?")

    # Esperar 2 segundos para dar tiempo al usuario a ingresar su elección
    sleep(2)

    # Utilizar la entrada de texto para buscar productos y presionar la tecla "Enter"
    driver.find_element("css selector", "#search-bar > div > form > input").send_keys(eleccion_usuario, Keys.ENTER)

    # Esperar 2 segundos para que se muestren los resultados de la búsqueda
    sleep(2)

    # Inicializar un diccionario para almacenar los resultados
    diccionario_resultados = {"nombre": [], "precio": [], "puntuacion": []}
    
    # Iterar sobre los primeros 5 resultados de búsqueda
    for i in range(1, 5):
        
        # Hacer clic en un resultado específico
        
        driver.find_element("css selector", f"#app > main > div.list-body.svelte-4gq2qk > section.listing-section.svelte-4gq2qk > div > div:nth-child({i})").click()

        sleep(3)

        # Recopilar el nombre del producto
        nombre = driver.find_element("css selector", "#app > main > article > div.vtmn-relative.vtmn-z-\[2\] > section > h1").text


        diccionario_resultados["nombre"].append(nombre)

        # Recopilar el precio del producto
        precio = driver.find_element("css selector","#app > main > article > div.vtmn-relative.vtmn-z-\[2\] > section > div.price-presentation.vtmn-flex.vtmn-flex-col.vtmn-items-start.vtmn-leading-tight.vtmn-mt-2.svelte-1wujduz > div.vtmn-flex.vtmn-flex-col.vtmn-items-start.vtmn-items-stretch.svelte-1wujduz > div > div.vtmn-flex.vtmn-flex-col.vtmn-justify-center.vtmn-items-end > span").text


        diccionario_resultados["precio"].append(precio)

        # Recopilar la puntuación del producto
        puntuacion = driver.find_element("xpath", '/html/body/div[2]/main/article/div[2]/section/div[3]/div/button/div/span[6]').text

        #elemento 1
        #(selector) #app > main > article > div.vtmn-relative.vtmn-z-\[2\] > section > div:nth-child(5) > div > button > div > span.vtmn-rating_comment--primary
        #           #app > main > article > div.vtmn-relative.vtmn-z-\[2\] > section > div:nth-child(5) > div > button > div > span.vtmn-rating_comment--primary

        #elemento 5
        #(selector) #app > main > article > div.vtmn-relative.vtmn-z-\[2\] > section > div:nth-child(4) > div > button > div > span.vtmn-rating_comment--primary
        
        #(xpath)//*[@id="app"]/main/article/div[2]/section/div[2]/div/button/div/span[6]
        
        #(full xpath) /html/body/div[2]/main/article/div[2]/section/div[2]/div/button/div/span[6]
        
        
        diccionario_resultados["puntuacion"].append(puntuacion)

        sleep(3)
        
        # Navegar de regreso a los resultados de búsqueda
        driver.back()

    # Cerrar el navegador
    driver.close()

    # Retornar el diccionario con los resultados
    return diccionario_resultados

In [3]:
# Ejemplo de uso:
resultados = buscar_productos_decathlon()
resultados

{'nombre': ['Zapatillas Caminar Flex Appeal 4.0 Mujer Blanco',
  'Zapatillas Deportivas Caminar Skechers Slip-ins Summits Hombre Azul Navy',
  'Zapatillas Caminar Skechers Slip Ins Summits Diamond Dream Mujer Azul Marino',
  'Zapatillas VL Court Adidas Niños Blanco Negro Gris'],
 'precio': ['44,99 €', '60,99 €', '57,99 €', '47,99 €'],
 'puntuacion': ['5/5', '4.5/5', '4.8/5', '4.9/5']}

In [4]:
# convertir a DataFrame
df_productos = pd.DataFrame(resultados)
df_productos

,nombre,precio,puntuacion
0,Zapatillas Caminar Flex Appeal 4.0 Mujer Blanco,"44,99 €",5/5
1,Zapatillas Deportivas Caminar Skechers Slip-in...,"60,99 €",4.5/5
2,Zapatillas Caminar Skechers Slip Ins Summits D...,"57,99 €",4.8/5
3,Zapatillas VL Court Adidas Niños Blanco Negro ...,"47,99 €",4.9/5
